In [1]:
import scanpy as sc
import pandas as pd
import infercnvpy as cnv

In [2]:
adata = sc.read_h5ad('/home/augusta/SSS_mount/insituCNV/data/WTx-CosMx_TVA/WTx-CosMx/adata_1970164_processed_metadata.h5ad')

In [3]:
adata

AnnData object with n_obs × n_vars = 609305 × 18946
    obs: 'Unnamed: 0', 'fov', 'Area', 'AspectRatio', 'CenterX_local_px', 'CenterY_local_px', 'Width', 'Height', 'Mean.PanCK', 'Max.PanCK', 'Mean.CD68_CK8_18', 'Max.CD68_CK8_18', 'Mean.CD298_B2M', 'Max.CD298_B2M', 'Mean.CD45', 'Max.CD45', 'Mean.DAPI', 'Max.DAPI', 'cell_id', 'version', 'Run_name', 'Run_Tissue_name', 'tissue', 'Panel', 'assay_type', 'slide_ID', 'CenterX_global_px', 'CenterY_global_px', 'cell_ID', 'unassignedTranscripts', 'nCount_RNA', 'nFeature_RNA', 'nCount_negprobes', 'nFeature_negprobes', 'nCount_falsecode', 'nFeature_falsecode', 'Area.um2', 'cell', 'sid', 'add_cell_id', 'add_ist', 'add_sub'
    var: 'Unnamed: 0', 'start', 'end', 'width', 'names'

#### Add genomic positions

In [4]:
dat = cnv.datasets.maynard2020_3k()
dat.var.loc[:, ["ensg", "chromosome", "start", "end"]].head()

,ensg,chromosome,start,end
symbol,,,,
AL645933.5,ENSG00000288587.1,chr6,31400702,31463705
AC010184.1,ENSG00000288585.1,chr3,141449745,141456434
AC023296.1,ENSG00000288580.1,chr8,2923568,2926689
AL117334.2,ENSG00000288577.1,chr20,3406380,3410036
AC107294.4,ENSG00000288576.1,chr3,184778723,184780720


In [5]:
adata=adata[:,adata.var.index.isin(dat.var['gene_name'])]

In [6]:
name2gid=dict(zip(dat.var['gene_name'],dat.var['gene_id']))
name2symbol=dict(zip(dat.var['gene_name'],dat.var.index))

In [7]:
adata.var['gene_id']=adata.var.index.map(name2gid)
adata.var['symbol']=adata.var.index.map(name2symbol)

/tmp/ipykernel_1967550/4285209568.py:1: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['gene_id']=adata.var.index.map(name2gid)


In [8]:
name2ensg=dict(zip(dat.var['gene_name'],dat.var['ensg']))
name2chromosome=dict(zip(dat.var['gene_name'],dat.var['chromosome']))
name2start=dict(zip(dat.var['gene_name'],dat.var['start']))
name2end=dict(zip(dat.var['gene_name'],dat.var['end']))

In [9]:
adata.var['ensg']=adata.var.index.map(name2ensg)
adata.var['chromosome']=adata.var.index.map(name2chromosome)
adata.var['start']=adata.var.index.map(name2start)
adata.var['end']=adata.var.index.map(name2end)

#### Pre-processing of data, PCA, UMAP

In [15]:
print(adata.layers['raw'])

  (0, 122)	1.0
  (0, 201)	1.0
  (0, 305)	1.0
  (0, 435)	1.0
  (0, 535)	1.0
  (0, 592)	1.0
  (0, 605)	1.0
  (0, 724)	1.0
  (0, 806)	1.0
  (0, 913)	1.0
  (0, 994)	1.0
  (0, 1094)	1.0
  (0, 1160)	1.0
  (0, 1201)	1.0
  (0, 1363)	1.0
  (0, 1407)	1.0
  (0, 1444)	1.0
  (0, 1485)	1.0
  (0, 1546)	1.0
  (0, 1597)	1.0
  (0, 1770)	1.0
  (0, 1774)	1.0
  (0, 1841)	1.0
  (0, 1909)	1.0
  (0, 1923)	1.0
  :	:
  (609304, 16562)	1.0
  (609304, 16621)	1.0
  (609304, 16628)	1.0
  (609304, 16651)	3.0
  (609304, 16662)	2.0
  (609304, 16856)	1.0
  (609304, 16907)	2.0
  (609304, 16958)	1.0
  (609304, 16987)	2.0
  (609304, 17026)	1.0
  (609304, 17074)	1.0
  (609304, 17095)	1.0
  (609304, 17128)	1.0
  (609304, 17225)	1.0
  (609304, 17411)	1.0
  (609304, 17573)	1.0
  (609304, 17736)	1.0
  (609304, 17741)	1.0
  (609304, 17799)	1.0
  (609304, 18222)	1.0
  (609304, 18227)	1.0
  (609304, 18339)	1.0
  (609304, 18388)	1.0
  (609304, 18448)	1.0
  (609304, 18535)	1.0


In [14]:
sc.pp.filter_cells(adata,min_counts=10)
adata.layers['raw']=adata.X.copy()

In [16]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [18]:
print(adata.layers['raw'])

  (0, 122)	1.0
  (0, 201)	1.0
  (0, 305)	1.0
  (0, 435)	1.0
  (0, 535)	1.0
  (0, 592)	1.0
  (0, 605)	1.0
  (0, 724)	1.0
  (0, 806)	1.0
  (0, 913)	1.0
  (0, 994)	1.0
  (0, 1094)	1.0
  (0, 1160)	1.0
  (0, 1201)	1.0
  (0, 1363)	1.0
  (0, 1407)	1.0
  (0, 1444)	1.0
  (0, 1485)	1.0
  (0, 1546)	1.0
  (0, 1597)	1.0
  (0, 1770)	1.0
  (0, 1774)	1.0
  (0, 1841)	1.0
  (0, 1909)	1.0
  (0, 1923)	1.0
  :	:
  (609304, 16562)	1.0
  (609304, 16621)	1.0
  (609304, 16628)	1.0
  (609304, 16651)	3.0
  (609304, 16662)	2.0
  (609304, 16856)	1.0
  (609304, 16907)	2.0
  (609304, 16958)	1.0
  (609304, 16987)	2.0
  (609304, 17026)	1.0
  (609304, 17074)	1.0
  (609304, 17095)	1.0
  (609304, 17128)	1.0
  (609304, 17225)	1.0
  (609304, 17411)	1.0
  (609304, 17573)	1.0
  (609304, 17736)	1.0
  (609304, 17741)	1.0
  (609304, 17799)	1.0
  (609304, 18222)	1.0
  (609304, 18227)	1.0
  (609304, 18339)	1.0
  (609304, 18388)	1.0
  (609304, 18448)	1.0
  (609304, 18535)	1.0


In [19]:
sc.pp.pca(adata)

In [20]:
sc.pp.neighbors(adata,n_pcs=40)

In [21]:
sc.tl.leiden(adata, n_iterations=2)

In [22]:
sc.tl.umap(adata)

#### Save the processed data

In [23]:
adata.write("/home/augusta/SSS_mount/insituCNV/data/WTx-CosMx_TVA/WTx-CosMx/adata_1970164_processed_metadata_processed.h5ad", compression = 'gzip')

... storing 'chromosome' as categorical
